<a href="https://colab.research.google.com/github/AlexKressner/WS24_Supply_Chain_Optimierung/blob/main/NetworkDesign_ErweiterungBeschaffung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Design Erweiterung um Beschaffungsmärkte

In [ ]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

Cloning into 'WS24_Supply_Chain_Optimierung'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 81 (delta 30), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 142.97 KiB | 4.47 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
! pip install -q pyscipopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 27.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pyscipopt import Model, quicksum

## Daten laden

In [ ]:
folder = "WS24_Supply_Chain_Optimierung/Daten/NetworkDesign"

In [ ]:
path = f"{folder}/NetworkDesign_Invest_Kapa.csv"

In [ ]:
kapa=pd.read_csv(path, sep=";")

In [ ]:
kapa.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [ ]:
path = f"{folder}/NetworkDesign_Produktionskosten.csv"

In [ ]:
produktionskosten=pd.read_csv(path, sep=";")

In [ ]:
produktionskosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [ ]:
path = f"{folder}/NetworkDesign_Nachfrage.csv"

In [ ]:
nachfrage=pd.read_csv(path, sep=";")

In [ ]:
nachfrage.head()

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


In [ ]:
path = f"{folder}/NetworkDesign_Beschaffungskosten.csv"

In [ ]:
beschaffung = pd.read_csv(path, sep=";")

In [ ]:
beschaffung.head()

,Beschaffungsmarkt,Vorprodukt,Beschaffungskosten,Kapazitäten
0,Afrika,Flasche,10,60
1,Afrika,Vitaminwasser,20,30
2,Asien,Flasche,12,100
3,Asien,Vitaminwasser,15,40
4,Europa,Flasche,15,60


## Optimierungsmodell

In [ ]:
scip = Model()

## Indexmengen

In [ ]:
I = produktionskosten["Produktionsstandort"].unique().tolist() # Menge der Produktionsstandorte

In [ ]:
J = nachfrage["Markt"].tolist() #Menge der Märkte

In [ ]:
A = kapa["Ausbaustufe"].unique().tolist() # Menge der Ausbaustufen an einem Standort

In [ ]:
M = beschaffung["Beschaffungsmarkt"].unique().tolist() # Menge der Beschaffungsmärkte

In [ ]:
VP = beschaffung["Vorprodukt"].unique().tolist() # Menge der Vorprodukte

## Entscheidungsvariablen

In [ ]:
# Definition der Entscheidungsvariablen
# X sind die Flussvariablen, d.h. Herstell- und Transportmenge von i nach j
X={}
for i in I:
  for j in J:
    X[i,j] = scip.addVar(vtype="C", name=f"{i},{j}")

In [ ]:
# Definition der Entscheidungsvariablen
# Y sind die Strukturvariablen, d.h. ob und mit welchen Kapazitäten ein Standort ausgebaut wird
Y={}
for i in I:
  for a in A:
    Y[i,a] = scip.addVar(vtype="B", name=f"{i},{a}")

In [ ]:
# Definition der Entscheidungsvariablen
# Z: Fluss von Vorprodukten vom Beschaffungsmarkt in den Produktionsstandort
Z={}
for m in M:
  for p in VP:
    for i in I:
      Z[m,p,i] = scip.addVar(vtype="C", name=f"{m},{p},{i}")

In [ ]:
print('Anzahl Entscheidungsvariablen =', len(scip.getVars()))

Anzahl Entscheidungsvariablen = 85


## Parameter

In [ ]:
# Herstell- und Transportkosten
produktionskosten.set_index(["Produktionsstandort","Markt"], inplace=True)

In [ ]:
cp = produktionskosten.to_dict("dict")["Produktionskosten"]

In [ ]:
cp

{('Charleston', 'USA'): 81,
 ('Charleston', 'Südamerika'): 92,
 ('Charleston', 'Europa'): 101,
 ('Charleston', 'Asien'): 130,
 ('Charleston', 'Afrika'): 115,
 ('Curitiba', 'USA'): 117,
 ('Curitiba', 'Südamerika'): 77,
 ('Curitiba', 'Europa'): 108,
 ('Curitiba', 'Asien'): 98,
 ('Curitiba', 'Afrika'): 100,
 ('Hamburg', 'USA'): 102,
 ('Hamburg', 'Südamerika'): 105,
 ('Hamburg', 'Europa'): 95,
 ('Hamburg', 'Asien'): 119,
 ('Hamburg', 'Afrika'): 111,
 ('Quanzhou', 'USA'): 115,
 ('Quanzhou', 'Südamerika'): 125,
 ('Quanzhou', 'Europa'): 90,
 ('Quanzhou', 'Asien'): 59,
 ('Quanzhou', 'Afrika'): 74,
 ('East London', 'USA'): 142,
 ('East London', 'Südamerika'): 100,
 ('East London', 'Europa'): 103,
 ('East London', 'Asien'): 105,
 ('East London', 'Afrika'): 71}

In [ ]:
# Nachfrage aus den Märtken

In [ ]:
nachfrage.set_index(["Markt"], inplace=True)

In [ ]:
d = nachfrage.to_dict("dict")["Nachfragemenge"]

In [ ]:
d

{'USA': 12, 'Südamerika': 8, 'Europa': 14, 'Asien': 16, 'Afrika': 7}

In [ ]:
# Kapazitäten der Ausbaustufen und Investitionskosten
kapa.set_index(["Produktionsstandort","Ausbaustufe"], inplace=True)

In [ ]:
pcap = kapa.to_dict("dict")["Kapazitäten"]

In [ ]:
pcap

{('Charleston', 'klein'): 10,
 ('Charleston', 'groß'): 20,
 ('Curitiba', 'klein'): 10,
 ('Curitiba', 'groß'): 20,
 ('Hamburg', 'klein'): 10,
 ('Hamburg', 'groß'): 20,
 ('Quanzhou', 'klein'): 10,
 ('Quanzhou', 'groß'): 20,
 ('East London', 'klein'): 10,
 ('East London', 'groß'): 20}

In [ ]:
cf = kapa.to_dict("dict")["Investitionen"]

In [ ]:
cf

{('Charleston', 'klein'): 6000,
 ('Charleston', 'groß'): 9000,
 ('Curitiba', 'klein'): 4500,
 ('Curitiba', 'groß'): 6750,
 ('Hamburg', 'klein'): 6500,
 ('Hamburg', 'groß'): 9750,
 ('Quanzhou', 'klein'): 4100,
 ('Quanzhou', 'groß'): 6150,
 ('East London', 'klein'): 4000,
 ('East London', 'groß'): 6000}

In [ ]:
# Beschaffungskosten und Kapazitäten
beschaffung.set_index(["Beschaffungsmarkt","Vorprodukt"], inplace=True)

In [ ]:
cb = beschaffung.to_dict("dict")["Beschaffungskosten"]

In [ ]:
cb

{('Afrika', 'Flasche'): 10,
 ('Afrika', 'Vitaminwasser'): 20,
 ('Asien', 'Flasche'): 12,
 ('Asien', 'Vitaminwasser'): 15,
 ('Europa', 'Flasche'): 15,
 ('Europa', 'Vitaminwasser'): 20,
 ('Südamerika', 'Flasche'): 8,
 ('Südamerika', 'Vitaminwasser'): 10,
 ('USA', 'Flasche'): 11,
 ('USA', 'Vitaminwasser'): 6}

In [ ]:
bcap = beschaffung.to_dict("dict")["Kapazitäten"]

In [ ]:
bcap

{('Afrika', 'Flasche'): 60,
 ('Afrika', 'Vitaminwasser'): 30,
 ('Asien', 'Flasche'): 100,
 ('Asien', 'Vitaminwasser'): 40,
 ('Europa', 'Flasche'): 60,
 ('Europa', 'Vitaminwasser'): 60,
 ('Südamerika', 'Flasche'): 40,
 ('Südamerika', 'Vitaminwasser'): 30,
 ('USA', 'Flasche'): 50,
 ('USA', 'Vitaminwasser'): 35}

## Zielfunktion

min $K=\sum_{i,j} c_{ij}* X_{ij} + \sum_{i,a} fc_{ia}* Y_{ia} + \sum_{m,p,i} bc_{mp}* Z_{mpi}$

In [ ]:
# Maximierung der gesamten Deckungsbeiträge
scip.setObjective(
    quicksum(cp[i,j]*X[i,j] for i in I for j in J) + #Produktionskosten
    quicksum(cf[i,a]*Y[i,a] for i in I for a in A) + #Invest
    quicksum(cb[m,p]*Z[m,p,i] for m in M for p in VP for i in I), #Beschaffungskosten
    sense="minimize"
    )

## Nebenbedingungen

## Optimierung

In [ ]:
scip.redirectOutput()

In [ ]:
scip.setRealParam('limits/time', 360.0)

In [ ]:
scip.optimize()
print(scip.getStatus())

presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 75 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 75 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 85 variables (10 bin, 0 int, 0 impl, 75 cont) and 30 constraints
     30 constraints of type <linear>
Presolving Time: 0.01

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
p 0.0s|     1 |     0 |    25 |     - |   locks|   0 |  85 |  30 |  30 |   0 |  0 |   0 |   0 | 0.000000e+00 | 6.795900e+04 |    Inf | unk

## Berechnung Lösung

In [ ]:
print('LÖSUNG:')
print('Zielfunktionswert (Kosten) =', scip.getObjVal())
print("\n")
print("PRODUKTIONS-MARKT-ZUTEILUNG")
for j in J:
  print(f"{j}:")
  for i in I:
    if scip.getVal(X[i,j])>0:
      print('\t', f'{i,j} =', round(scip.getVal(X[i,j])))
print("\n")
print("INVESTITIONEN")
for i in I:
  for a in A:
    if scip.getVal(Y[i,a])>0:
      print(f'{i,a} =', round(scip.getVal(Y[i,a])))
print("\n")
print("Beschaffungsmengen")
for i in I:
  for p in VP:
    for m in M:
      if scip.getVal(Z[m,p,i]):
        print(f'{i,p,m} =', round(scip.getVal(Z[m,p,i])))

LÖSUNG:
Zielfunktionswert (Kosten) = 24671.0


PRODUKTIONS-MARKT-ZUTEILUNG
USA:
	 ('Curitiba', 'USA') = 12
Südamerika:
	 ('Curitiba', 'Südamerika') = 8
Europa:
	 ('Quanzhou', 'Europa') = 4
	 ('East London', 'Europa') = 10
Asien:
	 ('Quanzhou', 'Asien') = 16
Afrika:
	 ('East London', 'Afrika') = 7


INVESTITIONEN
('Curitiba', 'groß') = 1
('Quanzhou', 'groß') = 1
('East London', 'groß') = 1


Beschaffungsmengen
('Curitiba', 'Flasche', 'Afrika') = 17
('Curitiba', 'Flasche', 'Südamerika') = 3
('Curitiba', 'Vitaminwasser', 'Südamerika') = 20
('Quanzhou', 'Flasche', 'Südamerika') = 20
('Quanzhou', 'Vitaminwasser', 'Südamerika') = 2
('Quanzhou', 'Vitaminwasser', 'USA') = 18
('East London', 'Flasche', 'Südamerika') = 17
('East London', 'Vitaminwasser', 'USA') = 17
